In [42]:
import ortools
from ortools.sat.python import cp_model

In [114]:
# Data.
num_nurses = 6
num_shifts = 3
num_days = 14
all_nurses = range(num_nurses)
all_shifts = range(num_shifts)
all_days = range(num_days)
#all_days = [[0,1,2,3,4,5,6],[7,8,9,10,11,12,13]]
#calendar = {0:{'week':0, 'month':0}, 1:{'week':0, 'month':0}, 2:{'week':0, 'month':0}, 3:{'week':0, 'month':0}, 4:{'week':0, 'month':0}, 5:{'week':0, 'month':0}, 6:{'week':0, 'month':0},
            #7:{'week':1, 'month':0}, 8:{'week':1, 'month':0}, 9:{'week':1, 'month':0}, 10:{'week':1, 'month':0}, 11:{'week':1, 'month':0}, 12:{'week':1, 'month':0}, 13:{'week':1, 'month':0}}

calendar = {'week':{0:[0,1,2,3,4,5,6], 1:[7,8,9,10,11,12,13]}, 'month':{0:[0,1,2,3,4,5,6,7,8,9,10,11,12,13]}}
required_nurses = [[3,1,1],[3,1,1],[3,1,1],[3,1,1],[3,1,1],[2,2,1],[2,2,1],[3,1,1],[3,1,1],[3,1,1],[3,1,1],[3,1,1],[2,2,1],[2,2,1]]
hourly_wage = [x*8 for x in [25,30,15,20,35,20]]
seniority_high = [0,1,0,0,1,1]

In [115]:
# Creates the model.
model = cp_model.CpModel()

# Creates shift variables.
# shifts[(n, d, s)]: nurse 'n' works shift 's' on day 'd'.
shifts = {}
for n in all_nurses:
    for d in all_days:
        for s in all_shifts:
            shifts[(n, d,
                    s)] = model.NewBoolVar(f'shift_n{n}d{d}s{s}') 

# Each shift is assigned to exactly one nurse in the schedule period.
for d in all_days:
    for s in all_shifts:
        model.Add(sum(shifts[(n, d, s)] for n in all_nurses) == required_nurses[d][s])

# Each nurse works at most one shift per day.
for n in all_nurses:
    for d in all_days:
        model.Add(sum(shifts[(n, d, s)] for s in all_shifts) <= 1)

# Each shift must have at least one seniority nurse per shift
for d in all_days:
    for s in all_shifts:
        model.Add(sum(shifts[(n, d, s)]*seniority_high[n] for n in all_nurses) == 1)

# max 3 nights per week
#for week in all_days:
#   for d in week:
#       model.Add(sum(shifts[(n, d, s)] for ) <= 3)

#5 shifts per week
for n in all_nurses:
    for week in calendar['week']:
                    model.Add(sum(shifts[(n, d, s)] for s in all_shifts for d in calendar['week'][week]) >= 4)
                    
# Objective Function
model.Minimize(
    sum(hourly_wage[n] * shifts[(n, d, s)] for n in all_nurses
        for d in all_days for s in all_shifts))

# Creates the solver and solve.
solver = cp_model.CpSolver()
status = solver.Solve(model)

In [116]:
if status == cp_model.OPTIMAL:
    print('Solution:')
    for d in all_days:
        print('Day', d)
        for n in all_nurses:
            for s in all_shifts:
                if solver.Value(shifts[(n,d,s)]) == 1:
                 print('  Nurse %i (seniority: %i) works shift %i (%i$)' % (n, seniority_high[n], s, hourly_wage[n]))
    print(f'Total wages = {solver.ObjectiveValue()}')
else:
    print('No optimal solution found !')


# Statistics.
print('\nStatistics')
print('  - conflicts: %i' % solver.NumConflicts())
print('  - branches : %i' % solver.NumBranches())
print('  - wall time: %f s' % solver.WallTime())

Solution:
Day 0
  Nurse 0 (seniority: 0) works shift 0 (200$)
  Nurse 1 (seniority: 1) works shift 1 (240$)
  Nurse 2 (seniority: 0) works shift 0 (120$)
  Nurse 4 (seniority: 1) works shift 2 (280$)
  Nurse 5 (seniority: 1) works shift 0 (160$)
Day 1
  Nurse 1 (seniority: 1) works shift 1 (240$)
  Nurse 2 (seniority: 0) works shift 0 (120$)
  Nurse 3 (seniority: 0) works shift 0 (160$)
  Nurse 4 (seniority: 1) works shift 2 (280$)
  Nurse 5 (seniority: 1) works shift 0 (160$)
Day 2
  Nurse 0 (seniority: 0) works shift 0 (200$)
  Nurse 1 (seniority: 1) works shift 2 (240$)
  Nurse 3 (seniority: 0) works shift 0 (160$)
  Nurse 4 (seniority: 1) works shift 1 (280$)
  Nurse 5 (seniority: 1) works shift 0 (160$)
Day 3
  Nurse 0 (seniority: 0) works shift 0 (200$)
  Nurse 1 (seniority: 1) works shift 2 (240$)
  Nurse 2 (seniority: 0) works shift 0 (120$)
  Nurse 4 (seniority: 1) works shift 1 (280$)
  Nurse 5 (seniority: 1) works shift 0 (160$)
Day 4
  Nurse 1 (seniority: 1) works shift 1 (